# Module 7: Advanced GitHub Features

**Estimated Time**: 4-5 hours | **Difficulty**: Advanced

## Learning Objectives

By the end of this module, you will be able to:

1. Master advanced Git commands (rebase, cherry-pick, stash, reflog)
2. Implement Git hooks for automation
3. Deploy websites using GitHub Pages
4. Integrate GitHub API for automation and data retrieval
5. Enable security features (Dependabot, code scanning)
6. Use GitHub Discussions and Wiki for documentation
7. Analyze repository insights and metrics
8. Master GitHub CLI (gh) for terminal workflows
9. Work with Git submodules and subtrees
10. Create and use repository templates

---

## Prerequisites

- Completion of Modules 1-6
- Solid understanding of Git basics
- Familiarity with GitHub workflows
- GitHub account with repository access

---

## Setup

Let's import the necessary libraries and set up our environment.

In [ ]:
import os
import subprocess
import git
from github import Github
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
from datetime import datetime, timedelta
import requests
from IPython.display import display, Markdown, HTML


# Helper function for running git commands
def run_git_command(command, cwd="."):
    """Execute a git command and return the output."""
    try:
        result = subprocess.run(
            command, shell=True, cwd=cwd, capture_output=True, text=True, check=False
        )
        return result.stdout.strip() if result.returncode == 0 else result.stderr.strip()
    except Exception as e:
        return f"Error: {str(e)}"


print("✓ Environment setup complete!")

---

## 1. Advanced Git Commands

### 1.1 Git Rebase: Rewriting History

**Rebase** rewrites commit history by moving or combining commits. It's powerful but should be used carefully.

#### When to Use Rebase:
- Clean up messy commit history before merging
- Keep a linear project history
- Incorporate upstream changes without merge commits

#### When NOT to Use Rebase:
- On public branches that others are using
- When you need to preserve exact commit history

#### Basic Rebase

In [ ]:
# Example: Rebase feature branch onto main
demo_commands = """
# Current situation: feature branch diverged from main
git checkout feature-branch
git rebase main

# This replays feature-branch commits on top of main
# Result: linear history without merge commits

# If conflicts occur:
# 1. Fix conflicts in files
# 2. git add <fixed-files>
# 3. git rebase --continue

# To abort rebase:
git rebase --abort
"""

print(demo_commands)

# Visualize rebase
display(
    Markdown(
        """
**Before Rebase:**
```
      C---D  feature-branch
     /
A---B---E---F  main
```

**After Rebase:**
```
A---B---E---F---C'---D'  feature-branch
            |
            main
```
(C' and D' are new commits with same changes but different hashes)
"""
    )
)

#### Interactive Rebase

Interactive rebase (`git rebase -i`) lets you edit, reorder, squash, or delete commits.

In [ ]:
# Interactive rebase example
display(
    Markdown(
        """
### Clean up last 3 commits:

```bash
git rebase -i HEAD~3
```

This opens an editor with:
```
pick abc1234 Add feature X
pick def5678 Fix typo
pick ghi9012 Update documentation
```

**Available Commands:**
- `pick` - Keep commit as is
- `reword` - Change commit message
- `edit` - Stop to amend commit
- `squash` - Combine with previous commit
- `fixup` - Like squash but discard message
- `drop` - Remove commit

**Example - Squash fixup commits:**
```
pick abc1234 Add feature X
fixup def5678 Fix typo
fixup ghi9012 Update documentation
```
Result: One clean commit with all changes
"""
    )
)

# Common interactive rebase workflow
workflow = """
# Before merging PR, clean up commits:
git checkout feature-branch
git rebase -i main

# In editor, squash WIP and fixup commits
# Save and close

# Force push (only for your own branches!)
git push --force-with-lease origin feature-branch
"""

print("Common workflow:")
print(workflow)

### 1.2 Git Cherry-Pick: Selective Commit Copying

Cherry-pick applies specific commits from one branch to another.

In [ ]:
# Cherry-pick examples
display(
    Markdown(
        """
### Use Cases:
- Apply a hotfix to multiple branches
- Recover specific commits from abandoned branches
- Backport features to release branches

### Basic Usage:
```bash
# Apply single commit
git cherry-pick abc1234

# Apply multiple commits
git cherry-pick abc1234 def5678

# Apply range of commits
git cherry-pick abc1234..def5678

# Cherry-pick without committing (to modify first)
git cherry-pick -n abc1234

# If conflicts occur:
# 1. Fix conflicts
# 2. git add <files>
# 3. git cherry-pick --continue

# To abort:
git cherry-pick --abort
```

### Example Scenario:
```bash
# You fixed a bug in develop branch
git log --oneline develop
# abc1234 Fix critical security bug

# Apply same fix to production branch
git checkout production
git cherry-pick abc1234
git push origin production
```
"""
    )
)

### 1.3 Git Stash: Temporary Storage

Stash saves your uncommitted changes temporarily, allowing you to switch branches without committing.

In [ ]:
# Git stash examples
stash_commands = """
# Save current changes
git stash
# or with message
git stash save "WIP: feature X implementation"

# List all stashes
git stash list
# stash@{0}: WIP on feature: abc1234 Add feature X
# stash@{1}: On main: def5678 Fix bug

# Apply most recent stash (keeps stash)
git stash apply

# Apply specific stash
git stash apply stash@{1}

# Apply and remove stash
git stash pop

# Show stash contents
git stash show
git stash show -p  # with diff

# Stash including untracked files
git stash -u

# Stash including ignored files
git stash -a

# Create branch from stash
git stash branch feature-from-stash stash@{0}

# Delete specific stash
git stash drop stash@{1}

# Clear all stashes
git stash clear
"""

print(stash_commands)

# Common workflow
display(
    Markdown(
        """
### Common Stash Workflow:

```bash
# Working on feature, need to fix urgent bug
git stash save "WIP: feature implementation"
git checkout main
git checkout -b hotfix/urgent-bug
# ... fix bug, commit, push ...
git checkout feature-branch
git stash pop  # Resume work
```
"""
    )
)

### 1.4 Git Reflog: Safety Net

Reflog records all HEAD movements, allowing you to recover "lost" commits.

In [ ]:
# Reflog examples
display(
    Markdown(
        """
### Git Reflog: Your Time Machine

Reflog keeps track of every change to HEAD for ~90 days (configurable).

```bash
# View reflog
git reflog
# Output:
# abc1234 HEAD@{0}: commit: Add feature
# def5678 HEAD@{1}: checkout: moving from main to feature
# ghi9012 HEAD@{2}: reset: moving to HEAD~1
# jkl3456 HEAD@{3}: commit: Wrong commit

# Recover lost commit
git checkout jkl3456  # Detached HEAD at lost commit
git checkout -b recovered-work  # Create branch

# Or directly reset to previous state
git reset --hard HEAD@{3}
```

### Recovery Scenarios:

**1. Accidental Reset:**
```bash
git reset --hard HEAD~3  # Oops, deleted 3 commits
git reflog  # Find commits
git reset --hard HEAD@{1}  # Restore
```

**2. Deleted Branch:**
```bash
git branch -D feature  # Oops
git reflog  # Find last commit on feature
git checkout -b feature abc1234  # Recreate branch
```

**3. Bad Rebase:**
```bash
git rebase main  # Conflicts, things went wrong
git rebase --abort  # Too late, already finished
git reflog  # Find pre-rebase state
git reset --hard HEAD@{5}  # Restore
```
"""
    )
)

---

## 2. Git Hooks: Automation

Git hooks are scripts that run automatically at specific points in the Git workflow.

### 2.1 Available Hooks

Hooks are stored in `.git/hooks/` directory.

In [ ]:
# Display available hooks
hooks_info = {
    "Client-Side Hooks": {
        "pre-commit": "Runs before commit is created",
        "prepare-commit-msg": "Runs before commit message editor opens",
        "commit-msg": "Validates commit message",
        "post-commit": "Runs after commit is created",
        "pre-rebase": "Runs before rebase",
        "post-checkout": "Runs after checkout",
        "post-merge": "Runs after merge",
        "pre-push": "Runs before push",
    },
    "Server-Side Hooks": {
        "pre-receive": "Runs when receiving push",
        "update": "Like pre-receive but per branch",
        "post-receive": "Runs after push is accepted",
    },
}

for category, hooks in hooks_info.items():
    print(f"\n{category}:")
    print("-" * 50)
    for hook, description in hooks.items():
        print(f"  {hook:20s} - {description}")

### 2.2 Creating Hooks

Let's create practical hooks for a Python project.

In [ ]:
# Example: pre-commit hook for Python code quality
pre_commit_hook = """#!/bin/bash
#
# Pre-commit hook: Run code quality checks
#

echo "Running pre-commit checks..."

# Check for Python files
python_files=$(git diff --cached --name-only --diff-filter=ACM | grep "\.py$")

if [ -z "$python_files" ]; then
    echo "No Python files to check"
    exit 0
fi

# Run Black formatter check
echo "Checking code formatting with Black..."
black --check $python_files
if [ $? -ne 0 ]; then
    echo "❌ Code formatting failed. Run: black ."
    exit 1
fi

# Run flake8 linter
echo "Running flake8 linter..."
flake8 $python_files
if [ $? -ne 0 ]; then
    echo "❌ Linting failed. Fix the issues above."
    exit 1
fi

# Run tests
echo "Running tests..."
pytest tests/
if [ $? -ne 0 ]; then
    echo "❌ Tests failed. Fix failing tests before committing."
    exit 1
fi

echo "✓ All pre-commit checks passed!"
exit 0
"""

print("Pre-commit hook for Python projects:")
print(pre_commit_hook)

# To install this hook:
display(
    Markdown(
        """
### Installing the Hook:

```bash
# Create hook file
cat > .git/hooks/pre-commit << 'EOF'
[paste hook content here]
EOF

# Make it executable
chmod +x .git/hooks/pre-commit

# Test it
git add .
git commit -m "Test commit"  # Hook will run

# Skip hook if needed (use sparingly!)
git commit --no-verify -m "Skip hooks"
```
"""
    )
)

In [ ]:
# Example: commit-msg hook to enforce commit message format
commit_msg_hook = """#!/bin/bash
#
# Commit-msg hook: Enforce conventional commits format
#

commit_msg_file=$1
commit_msg=$(cat "$commit_msg_file")

# Conventional commits pattern: type(scope): description
# Example: feat(auth): add login functionality
pattern="^(feat|fix|docs|style|refactor|test|chore)(\([a-z-]+\))?: .{10,}$"

if ! echo "$commit_msg" | grep -qE "$pattern"; then
    echo "❌ Invalid commit message format!"
    echo ""
    echo "Commit message must follow conventional commits:"
    echo "  type(scope): description"
    echo ""
    echo "Types: feat, fix, docs, style, refactor, test, chore"
    echo "Description: minimum 10 characters"
    echo ""
    echo "Examples:"
    echo "  feat(auth): add user authentication"
    echo "  fix(api): resolve null pointer exception"
    echo "  docs: update README with setup instructions"
    echo ""
    exit 1
fi

echo "✓ Commit message format is valid"
exit 0
"""

print("Commit-msg hook for conventional commits:")
print(commit_msg_hook)

In [ ]:
# Example: pre-push hook to prevent pushing to protected branches
pre_push_hook = """#!/bin/bash
#
# Pre-push hook: Prevent direct pushes to main/develop
#

protected_branches=("main" "master" "develop")
current_branch=$(git symbolic-ref HEAD | sed -e 's,.*/\\(.*\\),\\1,')

for branch in "${protected_branches[@]}"; do
    if [ "$branch" = "$current_branch" ]; then
        echo "❌ Direct push to $current_branch is not allowed!"
        echo ""
        echo "Please use a pull request instead:"
        echo "  1. Create a feature branch: git checkout -b feature/my-feature"
        echo "  2. Push feature branch: git push origin feature/my-feature"
        echo "  3. Create pull request on GitHub"
        echo ""
        echo "To override (NOT recommended): git push --no-verify"
        exit 1
    fi
done

echo "✓ Push allowed"
exit 0
"""

print("Pre-push hook to protect branches:")
print(pre_push_hook)

### 2.3 Sharing Hooks with Team

Hooks in `.git/hooks/` aren't tracked by Git. Use these approaches to share hooks:

In [ ]:
display(
    Markdown(
        """
### Method 1: Store hooks in repository

```bash
# Create hooks directory in repo
mkdir -p .githooks

# Move hooks there
mv .git/hooks/pre-commit .githooks/

# Configure Git to use this directory
git config core.hooksPath .githooks

# Add to version control
git add .githooks/
git commit -m "Add shared Git hooks"

# Team members run:
git config core.hooksPath .githooks
```

### Method 2: Setup script

Create `setup-hooks.sh`:
```bash
#!/bin/bash
# Copy hooks from repository to .git/hooks

echo "Installing Git hooks..."

cp .githooks/* .git/hooks/
chmod +x .git/hooks/*

echo "✓ Hooks installed!"
```

Team members run: `./setup-hooks.sh`

### Method 3: Use pre-commit framework

Install pre-commit tool:
```bash
pip install pre-commit
```

Create `.pre-commit-config.yaml`:
```yaml
repos:
  - repo: https://github.com/psf/black
    rev: 23.0.0
    hooks:
      - id: black
  
  - repo: https://github.com/PyCQA/flake8
    rev: 6.0.0
    hooks:
      - id: flake8
  
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.4.0
    hooks:
      - id: trailing-whitespace
      - id: end-of-file-fixer
      - id: check-yaml
      - id: check-added-large-files
```

Install hooks:
```bash
pre-commit install
```

This is the recommended approach for teams!
"""
    )
)

---

## 3. GitHub Pages: Free Hosting

GitHub Pages hosts static websites directly from your repository.

### 3.1 Types of GitHub Pages Sites

In [ ]:
display(
    Markdown(
        """
### 1. User/Organization Site
- **Repository name**: `username.github.io`
- **URL**: `https://username.github.io`
- **Source**: Main branch or `docs/` folder
- **Limit**: One per account

### 2. Project Site
- **Repository name**: Any name (e.g., `my-project`)
- **URL**: `https://username.github.io/my-project`
- **Source**: Main branch, `gh-pages` branch, or `docs/` folder
- **Limit**: Unlimited

### Setting Up GitHub Pages:

#### Method 1: Using main branch
```bash
# Create index.html in repository root
cat > index.html << 'EOF'
<!DOCTYPE html>
<html>
<head>
    <title>My GitHub Pages Site</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 50px auto;
            padding: 20px;
        }
    </style>
</head>
<body>
    <h1>Welcome to My Project</h1>
    <p>This site is hosted on GitHub Pages!</p>
</body>
</html>
EOF

git add index.html
git commit -m "Add GitHub Pages site"
git push
```

Then on GitHub:
1. Go to Settings → Pages
2. Source: Deploy from branch
3. Branch: main / (root)
4. Save

#### Method 2: Using docs/ folder
```bash
mkdir docs
cat > docs/index.html << 'EOF'
<!DOCTYPE html>
<html>
<head><title>Documentation</title></head>
<body><h1>Project Documentation</h1></body>
</html>
EOF

git add docs/
git commit -m "Add documentation site"
git push
```

On GitHub: Settings → Pages → Branch: main /docs

#### Method 3: Using gh-pages branch
```bash
# Create orphan branch (no history)
git checkout --orphan gh-pages
git rm -rf .

# Create site
echo "<h1>GitHub Pages</h1>" > index.html

git add index.html
git commit -m "Initialize GitHub Pages"
git push origin gh-pages

# Switch back to main
git checkout main
```

On GitHub: Settings → Pages → Branch: gh-pages
"""
    )
)

### 3.2 Using Jekyll with GitHub Pages

Jekyll is a static site generator that GitHub Pages supports natively.

In [ ]:
display(
    Markdown(
        """
### Quick Start with Jekyll:

1. **Create `_config.yml`:**
```yaml
title: My Awesome Project
description: A brief description of my project
theme: jekyll-theme-cayman
```

2. **Create `index.md` (Markdown instead of HTML):**
```markdown
---
layout: default
title: Home
---

# Welcome to My Project

This is my awesome project hosted on GitHub Pages with Jekyll.

## Features

- Feature 1
- Feature 2
- Feature 3

## Installation

```bash
pip install my-project
```
```

3. **Available Themes:**
```yaml
# In _config.yml
theme: jekyll-theme-minimal
# or
theme: jekyll-theme-cayman
# or
theme: jekyll-theme-architect
```

Browse themes: https://pages.github.com/themes/

4. **Using Remote Themes:**
```yaml
remote_theme: pages-themes/minimal@v0.2.0
plugins:
  - jekyll-remote-theme
```

### Advanced Jekyll Features:

**Blog Posts** (in `_posts/` directory):
```markdown
<!-- _posts/2024-01-15-my-first-post.md -->
---
layout: post
title: "My First Blog Post"
date: 2024-01-15
categories: [tutorial, github]
---

Content here...
```

**Custom Layouts** (in `_layouts/` directory):
```html
<!-- _layouts/default.html -->
<!DOCTYPE html>
<html>
<head>
    <title>{{ page.title }} | {{ site.title }}</title>
</head>
<body>
    <header>
        <h1>{{ site.title }}</h1>
    </header>
    <main>
        {{ content }}
    </main>
</body>
</html>
```
"""
    )
)

### 3.3 Deploying with GitHub Actions

In [ ]:
# Example: Deploy React app to GitHub Pages
deploy_workflow = """
name: Deploy to GitHub Pages

on:
  push:
    branches: [ main ]

permissions:
  contents: write

jobs:
  build-and-deploy:
    runs-on: ubuntu-latest
    
    steps:
    - name: Checkout
      uses: actions/checkout@v3
    
    - name: Setup Node
      uses: actions/setup-node@v3
      with:
        node-version: '18'
    
    - name: Install dependencies
      run: npm install
    
    - name: Build
      run: npm run build
    
    - name: Deploy
      uses: peaceiris/actions-gh-pages@v3
      with:
        github_token: ${{ secrets.GITHUB_TOKEN }}
        publish_dir: ./build
"""

print("GitHub Actions workflow for Pages deployment:")
print(deploy_workflow)
print("\nSave as: .github/workflows/deploy-pages.yml")

---

## 4. GitHub API Integration

Automate GitHub tasks using the API.

### 4.1 Authentication

In [ ]:
display(
    Markdown(
        """
### Setting Up GitHub API Access:

1. **Create Personal Access Token:**
   - Go to GitHub Settings → Developer settings → Personal access tokens → Tokens (classic)
   - Click "Generate new token (classic)"
   - Select scopes: `repo`, `user`, `workflow`
   - Copy token (you won't see it again!)

2. **Store Token Securely:**
```bash
# Create .env file (add to .gitignore!)
echo "GITHUB_TOKEN=your_token_here" > .env
echo ".env" >> .gitignore
```

3. **Load Token in Python:**
```python
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv('GITHUB_TOKEN')
```
"""
    )
)

# For this demo, we'll use environment variable
# DON'T hardcode tokens in notebooks!
github_token = os.getenv("GITHUB_TOKEN", "your_token_here")

if github_token == "your_token_here":
    print("⚠️  Set GITHUB_TOKEN environment variable to use API examples")
else:
    print("✓ GitHub token loaded")

### 4.2 Basic API Usage with PyGithub

In [ ]:
# Initialize GitHub API client
try:
    g = Github(github_token)
    user = g.get_user()
    print(f"Authenticated as: {user.login}")
    print(f"Public repos: {user.public_repos}")
    print(f"Followers: {user.followers}")
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Make sure GITHUB_TOKEN is set correctly")

In [ ]:
# Example: List repositories
def list_user_repos(username, limit=10):
    """List user's public repositories."""
    try:
        g = Github(github_token)
        user = g.get_user(username)
        repos = user.get_repos(sort="updated")[:limit]

        repo_data = []
        for repo in repos:
            repo_data.append(
                {
                    "Name": repo.name,
                    "Stars": repo.stargazers_count,
                    "Forks": repo.forks_count,
                    "Language": repo.language,
                    "Updated": repo.updated_at.strftime("%Y-%m-%d"),
                }
            )

        df = pd.DataFrame(repo_data)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None


# Example usage (replace with actual username)
# df = list_user_repos('torvalds', limit=5)
# if df is not None:
#     display(df)

print("Function defined: list_user_repos(username, limit=10)")

In [ ]:
# Example: Create repository
def create_repository(name, description="", private=False):
    """Create a new GitHub repository."""
    try:
        g = Github(github_token)
        user = g.get_user()

        repo = user.create_repo(
            name=name,
            description=description,
            private=private,
            auto_init=True,  # Initialize with README
            gitignore_template="Python",
        )

        print(f"✓ Repository created: {repo.html_url}")
        return repo
    except Exception as e:
        print(f"Error creating repository: {e}")
        return None


# Example usage:
# repo = create_repository(
#     name="test-api-repo",
#     description="Created via GitHub API",
#     private=True
# )

print("Function defined: create_repository(name, description, private)")

In [ ]:
# Example: Create issue
def create_issue(repo_name, title, body, labels=None):
    """Create an issue in a repository."""
    try:
        g = Github(github_token)
        user = g.get_user()
        repo = g.get_repo(f"{user.login}/{repo_name}")

        issue = repo.create_issue(title=title, body=body, labels=labels or [])

        print(f"✓ Issue created: {issue.html_url}")
        print(f"  Number: #{issue.number}")
        return issue
    except Exception as e:
        print(f"Error creating issue: {e}")
        return None


# Example usage:
# issue = create_issue(
#     repo_name="my-repo",
#     title="Bug: App crashes on startup",
#     body="## Description\n\nApp crashes when launched.\n\n## Steps to Reproduce\n\n1. Step 1\n2. Step 2",
#     labels=["bug", "high-priority"]
# )

print("Function defined: create_issue(repo_name, title, body, labels)")

In [ ]:
# Example: Analyze repository activity
def analyze_repo_activity(repo_full_name):
    """Analyze repository commits and contributors."""
    try:
        g = Github(github_token)
        repo = g.get_repo(repo_full_name)

        # Get commit activity
        commits = repo.get_commits()

        # Analyze last 100 commits
        commit_data = []
        for i, commit in enumerate(commits[:100]):
            commit_data.append(
                {
                    "date": commit.commit.author.date,
                    "author": commit.commit.author.name,
                    "message": commit.commit.message.split("\n")[0][:50],
                }
            )

        df = pd.DataFrame(commit_data)

        # Plot commits over time
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)

        # Daily commit counts
        daily_commits = df.resample("D").size()

        plt.figure(figsize=(12, 4))
        daily_commits.plot(kind="bar")
        plt.title(f"Commit Activity: {repo_full_name}")
        plt.xlabel("Date")
        plt.ylabel("Commits")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Top contributors
        print("\nTop Contributors:")
        contributor_counts = df["author"].value_counts().head(10)
        print(contributor_counts)

        return df
    except Exception as e:
        print(f"Error analyzing repository: {e}")
        return None


# Example usage:
# df = analyze_repo_activity('python/cpython')

print("Function defined: analyze_repo_activity(repo_full_name)")

### 4.3 Using REST API Directly

In [ ]:
# Example: Using requests library for GitHub API
def get_repo_info_rest(owner, repo):
    """Get repository information using REST API."""
    url = f"https://api.github.com/repos/{owner}/{repo}"
    headers = {"Authorization": f"token {github_token}", "Accept": "application/vnd.github.v3+json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        print(f"Repository: {data['full_name']}")
        print(f"Description: {data['description']}")
        print(f"Stars: {data['stargazers_count']}")
        print(f"Forks: {data['forks_count']}")
        print(f"Open Issues: {data['open_issues_count']}")
        print(f"Created: {data['created_at']}")
        return data
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
        return None


# Example usage:
# repo_info = get_repo_info_rest('facebook', 'react')

print("Function defined: get_repo_info_rest(owner, repo)")

---

## 5. Security Features

GitHub provides several security features to protect your repositories.

### 5.1 Dependabot: Automated Dependency Updates

In [ ]:
display(
    Markdown(
        """
### Enabling Dependabot:

1. **Enable in Repository Settings:**
   - Settings → Security & analysis
   - Enable "Dependabot alerts"
   - Enable "Dependabot security updates"
   - Enable "Dependabot version updates" (optional)

2. **Configure Dependabot** (`.github/dependabot.yml`):

```yaml
version: 2
updates:
  # Python dependencies
  - package-ecosystem: "pip"
    directory: "/"
    schedule:
      interval: "weekly"
      day: "monday"
      time: "09:00"
    open-pull-requests-limit: 10
    reviewers:
      - "your-username"
    assignees:
      - "your-username"
    labels:
      - "dependencies"
      - "python"
    
  # npm dependencies
  - package-ecosystem: "npm"
    directory: "/"
    schedule:
      interval: "weekly"
    versioning-strategy: increase
    
  # GitHub Actions
  - package-ecosystem: "github-actions"
    directory: "/"
    schedule:
      interval: "weekly"
```

### What Dependabot Does:
- Scans dependencies for security vulnerabilities
- Creates pull requests to update vulnerable dependencies
- Keeps dependencies up to date
- Provides security advisories

### Handling Dependabot PRs:
```bash
# Review PR on GitHub
# Check CI/CD passes
# Merge if tests pass

# Or update locally:
git fetch origin
git checkout dependabot/pip/requests-2.31.0
# Run tests
pytest
# If OK, merge on GitHub
```
"""
    )
)

### 5.2 Code Scanning with CodeQL

In [ ]:
# CodeQL workflow for code scanning
codeql_workflow = """
name: "CodeQL Security Scan"

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  schedule:
    - cron: '0 0 * * 1'  # Weekly on Monday

jobs:
  analyze:
    name: Analyze
    runs-on: ubuntu-latest
    permissions:
      security-events: write
      actions: read
      contents: read

    strategy:
      fail-fast: false
      matrix:
        language: [ 'python', 'javascript' ]

    steps:
    - name: Checkout repository
      uses: actions/checkout@v3

    - name: Initialize CodeQL
      uses: github/codeql-action/init@v2
      with:
        languages: ${{ matrix.language }}
        queries: security-extended

    - name: Autobuild
      uses: github/codeql-action/autobuild@v2

    - name: Perform CodeQL Analysis
      uses: github/codeql-action/analyze@v2
      with:
        category: "/language:${{matrix.language}}"
"""

print("CodeQL Security Scanning Workflow:")
print(codeql_workflow)
print("\nSave as: .github/workflows/codeql.yml")

display(
    Markdown(
        """
### What CodeQL Detects:
- SQL injection
- Cross-site scripting (XSS)
- Path traversal
- Command injection
- Insecure deserialization
- Use of hardcoded credentials
- And many more...

### Viewing Results:
- Security tab → Code scanning alerts
- Each alert includes:
  - Severity level
  - Affected code location
  - Remediation guidance
  - CVSS score
"""
    )
)

### 5.3 Secret Scanning

In [ ]:
display(
    Markdown(
        """
### GitHub Secret Scanning:

GitHub automatically scans for exposed secrets:
- API keys
- Access tokens
- Private keys
- Database connection strings
- Cloud provider credentials

### Enabling Secret Scanning:
1. Settings → Security & analysis
2. Enable "Secret scanning"
3. Enable "Push protection" (prevents pushing secrets)

### What to Do if Secret is Exposed:

1. **Immediately Revoke the Secret:**
   - Revoke API key/token
   - Change password
   - Rotate credentials

2. **Remove from History:**
```bash
# Use BFG Repo-Cleaner or git-filter-repo
# Install git-filter-repo:
pip install git-filter-repo

# Remove file from history
git filter-repo --path secrets.txt --invert-paths

# Force push (coordinate with team!)
git push --force-with-lease
```

3. **Use .gitignore to Prevent Future Leaks:**
```gitignore
# Environment files
.env
.env.local
.env.*.local

# Credentials
secrets/
*.pem
*.key
credentials.json
config/secrets.yml

# Cloud provider
.aws/
.gcp/
```

### Best Practices:
- Use environment variables
- Use secret management tools (AWS Secrets Manager, HashiCorp Vault)
- Never commit `.env` files
- Use GitHub Secrets for Actions
- Regularly rotate credentials
- Enable push protection
"""
    )
)

---

## 6. GitHub CLI (gh)

The GitHub CLI brings GitHub to your terminal.

In [ ]:
display(
    Markdown(
        """
### Installing GitHub CLI:

**macOS:**
```bash
brew install gh
```

**Windows:**
```bash
winget install GitHub.cli
# or
choco install gh
```

**Linux:**
```bash
# Debian/Ubuntu
sudo apt install gh

# Fedora
sudo dnf install gh
```

### Authentication:
```bash
gh auth login
# Follow prompts to authenticate
```
"""
    )
)

In [ ]:
# GitHub CLI common commands
gh_commands = """
### Repository Operations

# Clone repository
gh repo clone owner/repo

# Create repository
gh repo create my-new-repo --public --clone
gh repo create my-org/my-repo --private

# View repository
gh repo view
gh repo view owner/repo --web  # Open in browser

# Fork repository
gh repo fork owner/repo --clone

# List repositories
gh repo list
gh repo list owner --limit 20


### Pull Request Operations

# Create pull request
gh pr create --title "Add feature" --body "Description"
gh pr create --fill  # Use commit messages

# List pull requests
gh pr list
gh pr list --state open
gh pr list --assignee "@me"

# View pull request
gh pr view 123
gh pr view 123 --web

# Checkout pull request
gh pr checkout 123

# Review pull request
gh pr review 123 --approve
gh pr review 123 --request-changes --body "Needs work"
gh pr review 123 --comment --body "Looks good"

# Merge pull request
gh pr merge 123 --squash
gh pr merge 123 --rebase
gh pr merge 123 --merge

# Close pull request
gh pr close 123


### Issue Operations

# Create issue
gh issue create --title "Bug found" --body "Details..."
gh issue create --label bug --label high-priority

# List issues
gh issue list
gh issue list --state open
gh issue list --assignee "@me"
gh issue list --label bug

# View issue
gh issue view 456

# Close issue
gh issue close 456
gh issue close 456 --comment "Fixed in PR #123"

# Reopen issue
gh issue reopen 456


### Workflow Operations

# List workflows
gh workflow list

# View workflow runs
gh run list
gh run list --workflow=ci.yml

# View specific run
gh run view 789
gh run view 789 --log

# Re-run workflow
gh run rerun 789

# Watch workflow
gh run watch


### Release Operations

# Create release
gh release create v1.0.0 --title "Version 1.0.0" --notes "Release notes"
gh release create v1.0.0 ./dist/*.zip  # With assets

# List releases
gh release list

# View release
gh release view v1.0.0

# Download release assets
gh release download v1.0.0


### Gist Operations

# Create gist
gh gist create file.py --public
gh gist create file.py --desc "Description" --private

# List gists
gh gist list

# View gist
gh gist view abc123


### Advanced Usage

# Use jq for JSON processing
gh api repos/:owner/:repo | jq '.stargazers_count'

# Custom API calls
gh api /repos/owner/repo/issues
gh api -X POST /repos/owner/repo/issues -f title="New issue"

# Aliases
gh alias set bugs 'issue list --label=bug'
gh bugs  # Run alias
"""

print(gh_commands)

---

## 7. Git Submodules and Subtrees

Manage external dependencies as part of your repository.

### 7.1 Git Submodules

In [ ]:
display(
    Markdown(
        """
### What are Submodules?
Submodules allow you to keep a Git repository as a subdirectory of another Git repository.

### Adding Submodules:
```bash
# Add submodule
git submodule add https://github.com/user/repo.git path/to/submodule

# Example: Add a library
git submodule add https://github.com/user/awesome-lib.git libs/awesome-lib

# Commit the changes
git commit -m "Add awesome-lib submodule"
```

### Cloning Repository with Submodules:
```bash
# Clone with submodules
git clone --recurse-submodules https://github.com/user/project.git

# Or if already cloned:
git submodule init
git submodule update

# Or in one command:
git submodule update --init --recursive
```

### Updating Submodules:
```bash
# Update to latest commit
cd path/to/submodule
git pull origin main
cd ../..
git add path/to/submodule
git commit -m "Update submodule to latest"

# Or update all submodules:
git submodule update --remote --merge
```

### Removing Submodules:
```bash
# Remove submodule
git submodule deinit path/to/submodule
git rm path/to/submodule
rm -rf .git/modules/path/to/submodule
git commit -m "Remove submodule"
```

### Pros and Cons:

**Pros:**
- Pin to specific commit
- Clear separation between projects
- Shared code across repos

**Cons:**
- Complex to manage
- Easy to forget to update
- Requires extra steps when cloning
"""
    )
)

### 7.2 Git Subtrees

In [ ]:
display(
    Markdown(
        """
### What are Subtrees?
Subtrees allow you to nest one repository inside another as a subdirectory, but with the content copied in.

### Adding Subtrees:
```bash
# Add remote
git remote add lib-remote https://github.com/user/awesome-lib.git

# Add subtree
git subtree add --prefix=libs/awesome-lib lib-remote main --squash
```

### Updating Subtrees:
```bash
# Pull updates from subtree
git subtree pull --prefix=libs/awesome-lib lib-remote main --squash
```

### Contributing Back to Subtree:
```bash
# Push changes back to original repo
git subtree push --prefix=libs/awesome-lib lib-remote feature-branch
```

### Pros and Cons:

**Pros:**
- Simpler for cloning (no special commands)
- All code is in one repository
- No .gitmodules file to manage

**Cons:**
- Larger repository size
- History can become cluttered
- Less clear separation

### Submodules vs Subtrees:

| Feature | Submodules | Subtrees |
|---------|-----------|----------|
| Clone command | Special | Standard |
| Repository size | Smaller | Larger |
| Complexity | Higher | Lower |
| Version control | Pin to commit | Copy code |
| Use case | External dependencies | Vendor code |
"""
    )
)

---

## 8. Repository Templates

Create reusable repository templates for consistent project setup.

In [ ]:
display(
    Markdown(
        """
### Creating a Template Repository:

1. **Create repository on GitHub**
2. **Go to Settings**
3. **Check "Template repository"**
4. **Save**

### Using a Template:

**On GitHub:**
1. Go to template repository
2. Click "Use this template"
3. Name your new repository
4. Create repository

**Using GitHub CLI:**
```bash
gh repo create my-new-project --template owner/template-repo
```

### Example: Python Project Template

**Repository structure:**
```
python-project-template/
├── .github/
│   ├── workflows/
│   │   ├── ci.yml
│   │   └── publish.yml
│   ├── ISSUE_TEMPLATE/
│   │   ├── bug_report.md
│   │   └── feature_request.md
│   └── pull_request_template.md
├── src/
│   └── __init__.py
├── tests/
│   └── __init__.py
├── docs/
│   └── README.md
├── .gitignore
├── .pre-commit-config.yaml
├── README.md
├── requirements.txt
├── requirements-dev.txt
├── setup.py
├── LICENSE
└── pyproject.toml
```

### Template README.md:
```markdown
# Project Name

> Replace this with your project description

## Installation

```bash
pip install -r requirements.txt
```

## Usage

```python
# Add usage examples
```

## Development

```bash
# Install dev dependencies
pip install -r requirements-dev.txt

# Run tests
pytest

# Run linters
flake8 src/
black src/
```

## License

MIT
```

### Popular Template Repositories:

- Python: https://github.com/rochacbruno/python-project-template
- React: https://github.com/facebook/create-react-app
- Django: https://github.com/cookiecutter/cookiecutter-django
- FastAPI: https://github.com/tiangolo/full-stack-fastapi-template

### Best Practices for Templates:

1. Include comprehensive README with instructions
2. Add CI/CD workflows
3. Include issue and PR templates
4. Add .gitignore for your language/framework
5. Include license file
6. Add pre-commit hooks configuration
7. Include documentation structure
8. Add example code/tests
9. Keep dependencies minimal and up-to-date
10. Document what to customize after using template
"""
    )
)

---

## 9. Advanced Features Quick Reference

In [ ]:
# Create comprehensive quick reference
quick_reference = """
╔════════════════════════════════════════════════════════════════╗
║          ADVANCED GIT & GITHUB QUICK REFERENCE                 ║
╚════════════════════════════════════════════════════════════════╝

ADVANCED GIT COMMANDS
═══════════════════════
Rebase:
  git rebase main                    Rebase current branch onto main
  git rebase -i HEAD~3               Interactive rebase last 3 commits
  git rebase --abort                 Cancel rebase

Cherry-pick:
  git cherry-pick abc1234            Apply specific commit
  git cherry-pick abc..def           Apply range of commits

Stash:
  git stash                          Save current changes
  git stash list                     List all stashes
  git stash pop                      Apply and remove stash
  git stash apply stash@{1}          Apply specific stash

Reflog:
  git reflog                         View HEAD history
  git reset --hard HEAD@{2}          Restore to previous state

GITHUB CLI (gh)
═══════════════
Auth:
  gh auth login                      Authenticate

Repository:
  gh repo create name --public       Create repository
  gh repo clone owner/repo           Clone repository
  gh repo fork owner/repo            Fork repository

Pull Requests:
  gh pr create --fill                Create PR
  gh pr list                         List PRs
  gh pr checkout 123                 Checkout PR
  gh pr merge 123 --squash           Merge PR

Issues:
  gh issue create                    Create issue
  gh issue list --label bug          List issues
  gh issue close 456                 Close issue

GIT HOOKS
═════════
Location: .git/hooks/ or .githooks/

Common hooks:
  pre-commit                         Before commit
  commit-msg                         Validate commit message
  pre-push                           Before push

Make executable: chmod +x .git/hooks/pre-commit

GITHUB PAGES
════════════
Deploy methods:
  1. Main branch (root or /docs)
  2. gh-pages branch
  3. GitHub Actions workflow

URL formats:
  User site: https://username.github.io
  Project:   https://username.github.io/project

SECURITY
════════
Dependabot:
  Config: .github/dependabot.yml
  Auto updates dependencies

CodeQL:
  Workflow: .github/workflows/codeql.yml
  Scans for security vulnerabilities

Secret Scanning:
  Enable in Settings → Security & analysis
  Enable push protection

SUBMODULES
══════════
  git submodule add URL path         Add submodule
  git clone --recurse-submodules     Clone with submodules
  git submodule update --init        Initialize submodules
  git submodule update --remote      Update to latest

SUBTREES
════════
  git subtree add --prefix=path URL  Add subtree
  git subtree pull --prefix=path     Update subtree
  git subtree push --prefix=path     Push changes back

TIPS
════
• Use --force-with-lease instead of --force
• Never rebase public branches
• Keep commit history clean with interactive rebase
• Use reflog to recover from mistakes
• Enable security features (Dependabot, CodeQL, Secret scanning)
• Use pre-commit hooks for code quality
• Protect main/master branches in Settings
• Use GitHub CLI for faster workflows
"""

print(quick_reference)

---

## 10. Practice Exercises

### Exercise 1: Interactive Rebase

**Task**: Clean up a messy commit history

1. Create a feature branch with 5 commits (including some "WIP" and "fix typo" commits)
2. Use interactive rebase to squash fixup commits
3. Reword commit messages to be more descriptive
4. Force push to your branch

**Commands to practice**:
```bash
git checkout -b feature/clean-history
# ... make commits ...
git rebase -i HEAD~5
git push --force-with-lease
```

### Exercise 2: Set Up Git Hooks

**Task**: Create a pre-commit hook for Python code quality

1. Create `.githooks/pre-commit` that:
   - Runs `black --check` on Python files
   - Runs `flake8` linter
   - Prevents commit if checks fail
2. Configure repository to use `.githooks`
3. Test the hook by trying to commit poorly formatted code

**Bonus**: Install `pre-commit` framework and configure it with `.pre-commit-config.yaml`

### Exercise 3: Deploy to GitHub Pages

**Task**: Create and deploy a simple website

1. Create a new repository (or use existing)
2. Create `index.html` with a simple website
3. Add CSS styling
4. Deploy using GitHub Pages
5. Add a custom domain (optional)

**Bonus**: Use Jekyll with a theme

### Exercise 4: GitHub API Automation

**Task**: Create a Python script to analyze repositories

1. Use PyGithub to:
   - List your repositories
   - Find repositories with open issues
   - Create a report of commit activity
2. Visualize the data with matplotlib
3. Save results to CSV

**Bonus**: Create a dashboard with plotly or streamlit

### Exercise 5: Security Setup

**Task**: Enable all security features

1. Create `.github/dependabot.yml`
2. Set up CodeQL workflow
3. Enable secret scanning and push protection
4. Add security policy (SECURITY.md)
5. Configure branch protection rules

**Success criteria**: All security features enabled and green checkmarks in Security tab

### Exercise 6: Master GitHub CLI

**Task**: Complete common workflows using only `gh`

1. Create a new repository
2. Create an issue
3. Create a branch and make changes
4. Create a pull request
5. Review and merge the PR
6. Create a release

**All using `gh` commands - no GitHub web interface!**

### Exercise 7: Repository Template

**Task**: Create a reusable project template

1. Create a new repository with:
   - Directory structure for Python/Node.js/etc
   - CI/CD workflows
   - Issue and PR templates
   - Pre-commit hooks
   - README template
   - License
2. Mark as template repository
3. Test by creating a new project from template

**Goal**: Complete template ready for team use

---

## Summary

In this module, you learned:

1. **Advanced Git Commands**:
   - Rebase (regular and interactive) for clean history
   - Cherry-pick for selective commits
   - Stash for temporary storage
   - Reflog for recovering lost commits

2. **Git Hooks**:
   - Client-side and server-side hooks
   - Pre-commit, commit-msg, pre-push examples
   - Sharing hooks with team
   - Using pre-commit framework

3. **GitHub Pages**:
   - Deploying static websites
   - Using Jekyll for site generation
   - GitHub Actions deployment

4. **GitHub API**:
   - PyGithub for automation
   - Creating repos, issues, PRs programmatically
   - Analyzing repository data

5. **Security Features**:
   - Dependabot for dependency updates
   - CodeQL for code scanning
   - Secret scanning and push protection

6. **GitHub CLI**:
   - Terminal-based GitHub workflows
   - Repository, PR, issue operations
   - Workflow and release management

7. **Submodules and Subtrees**:
   - Managing external dependencies
   - When to use each approach

8. **Repository Templates**:
   - Creating reusable project structures
   - Best practices for templates

### Next Steps

- Complete Module 8: Final Project to apply all skills
- Explore GitHub Marketplace for useful actions and apps
- Contribute to open source projects
- Build your own automation tools with GitHub API
- Create your own repository templates for common project types

### Additional Resources

- [Git Documentation](https://git-scm.com/doc)
- [GitHub Docs - Advanced Features](https://docs.github.com/en)
- [PyGithub Documentation](https://pygithub.readthedocs.io/)
- [GitHub CLI Manual](https://cli.github.com/manual/)
- [Pre-commit Framework](https://pre-commit.com/)
- [Jekyll Documentation](https://jekyllrb.com/docs/)

**Congratulations on completing Module 7!** 🎉

You now have advanced Git and GitHub skills used by professional developers!